In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [2]:
df = pd.read_csv('Google-Playstore-32K.csv')

In [3]:
pd.set_option('display.max_rows', 10)
df.columns = df.columns.str.replace(' ', '')
df

,AppName,Category,Rating,Reviews,Installs,Size,Price,ContentRating,LastUpdated,MinimumVersion,LatestVersion
0,DoorDash - Food Delivery,FOOD_AND_DRINK,4.548561573,305034.0,"5,000,000+",Varies with device,0,Everyone,"March 29, 2019",Varies with device,Varies with device
1,TripAdvisor Hotels Flights Restaurants Attract...,TRAVEL_AND_LOCAL,4.400671482,1207922.0,"100,000,000+",Varies with device,0,Everyone,"March 29, 2019",Varies with device,Varies with device
2,Peapod,SHOPPING,3.656329393,1967.0,"100,000+",1.4M,0,Everyone,"September 20, 2018",5.0 and up,2.2.0
3,foodpanda - Local Food Delivery,FOOD_AND_DRINK,4.107232571,389154.0,"10,000,000+",16M,0,Everyone,"March 22, 2019",4.2 and up,4.18.2
4,My CookBook Pro (Ad Free),FOOD_AND_DRINK,4.647752285,2291.0,"10,000+",Varies with device,$5.99,Everyone,"April 1, 2019",Varies with device,Varies with device
...,...,...,...,...,...,...,...,...,...,...,...
31995,KG Newsroom,NEWS_AND_MAGAZINES,4.5,8.0,50+,9.6M,0,Everyone,"March 28, 2019",4.1 and up,1.1.15
31996,Dedi App 19,GAME_SIMULATION,3.521126747,71.0,"10,000+",6.5M,0,Everyone,"January 22, 2019",4.4 and up,1.0.2274
31997,Dedi App 17,GAME_SIMULATION,3.994588852,924.0,"100,000+",7.9M,0,Everyone,"February 17, 2017",4.0.3 and up,1.1.2043
31998,Right Angle Triangle Solver,TOOLS,4.25,1796.0,"100,000+",6.8M,0,Everyone,"December 15, 2018",4.1 and up,2.0.5


## Drop columns we don't need

In [4]:
df.drop(['MinimumVersion', 'LatestVersion'], axis=1, inplace=True)

## Drop rows containing NA

In [5]:
df.isna().sum()

AppName          0
Category         0
Rating           0
Reviews          1
Installs         0
Size             0
Price            0
ContentRating    0
LastUpdated      0
dtype: int64

In [6]:
df = pd.DataFrame.dropna(df)

## Remove duplicate apps

In [7]:
df.duplicated().sum()

1

In [8]:
df = df.drop_duplicates(keep='first').reset_index().drop('index',axis=1)
df.duplicated().sum()

0

## Clean 'ContentRating' column

In [9]:
df['ContentRating'].value_counts()

Everyone           27454
Teen                2860
Everyone 10+        1054
Mature 17+           622
Unrated                3
Adults only 18+        3
$0.99                  1
0                      1
Name: ContentRating, dtype: int64

In [10]:
df.loc[df['ContentRating'] == '$0.99']

,AppName,Category,Rating,Reviews,Installs,Size,Price,ContentRating,LastUpdated
13503,Never have I ever 18+,),GAME_STRATEGY,4.0,6,100+,2.4M,$0.99,Mature 17+


In [11]:
df.loc[df['ContentRating'] == '100,000+']

,AppName,Category,Rating,Reviews,Installs,Size,Price,ContentRating,LastUpdated


In [12]:
df.loc[df['ContentRating'] == '0']

,AppName,Category,Rating,Reviews,Installs,Size,Price,ContentRating,LastUpdated
23456,Israel News,Channel 2 News,NEWS_AND_MAGAZINES,3.857799,11976,"1,000,000+",Varies with device,0,Everyone 10+


Above, we can see that the data was shifted within the row. Since we cannot properly determine which data belongs to which app, we will drop the above rows.
#### Drop the rows where the 'shift' happened

In [13]:
df.drop(df[(df['ContentRating'] == '$0.99') | (df['ContentRating'] == '100,000+') 
           | (df['ContentRating'] == '0')].index, inplace = True)

## Clean 'Size' column, convert all to kilobytes

In [14]:
df['Size'].value_counts()

Varies with device    3195
11M                    859
12M                    711
13M                    616
14M                    598
                      ... 
113M                     1
531k                     1
565k                     1
1,024k                   1
327k                     1
Name: Size, Length: 576, dtype: int64

In [15]:
df.drop(df[df['Size'] == 'Varies with device'].index, inplace=True)

In [16]:
def convert_size(x): #convert to kilobytes
    unit = x[-1]
    
    if(unit == 'M'):
        value = float(x[0:-1])
        value = value*1024
    elif(unit == 'k'):
        value = x[0:-1]
        value = value.replace(',', '')
        value = float(value)
    
    return value

df['Size'] = df.apply(lambda row : convert_size(row['Size']), axis = 1)

## Clean 'Category' column

In [17]:
df['Category'].unique()

array(['SHOPPING', 'FOOD_AND_DRINK', 'TRAVEL_AND_LOCAL', 'LIFESTYLE',
       'GAME_ACTION', 'GAME_CASUAL', 'GAME_ROLE_PLAYING', 'GAME_PUZZLE',
       'GAME_ADVENTURE', 'GAME_ARCADE', 'GAME_STRATEGY', 'GAME_SPORTS',
       'GAME_SIMULATION', 'GAME_MUSIC', 'MUSIC_AND_AUDIO', 'FINANCE',
       'EVENTS', 'ENTERTAINMENT', 'EDUCATION', 'GAME_EDUCATIONAL',
       'BOOKS_AND_REFERENCE', 'PHOTOGRAPHY', 'VIDEO_PLAYERS',
       'NEWS_AND_MAGAZINES', 'GAME_WORD', 'ART_AND_DESIGN', 'GAME_TRIVIA',
       'GAME_BOARD', 'BUSINESS', 'PRODUCTIVITY', 'COMMUNICATION',
       'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'BEAUTY', 'SOCIAL',
       'MAPS_AND_NAVIGATION', 'GAME_CASINO', 'GAME_CARD', 'SPORTS',
       'AUTO_AND_VEHICLES', 'GAME_RACING', 'TOOLS', 'PERSONALIZATION',
       'LIBRARIES_AND_DEMO', 'COMICS', 'DATING', 'WEATHER', 'PARENTING',
       'MEDICAL'], dtype=object)

In [18]:
game_cat = ['GAME_ACTION', 'GAME_CASUAL', 'GAME_ROLE_PLAYING', 'GAME_PUZZLE',
       'GAME_ADVENTURE', 'GAME_ARCADE', 'GAME_STRATEGY', 'GAME_SPORTS',
       'GAME_SIMULATION', 'GAME_MUSIC', 'GAME_EDUCATIONAL', 'GAME_WORD', 'GAME_TRIVIA',
       'GAME_BOARD', 'GAME_CASINO', 'GAME_CARD', 'GAME_RACING']

df.loc[df['Category'].isin(game_cat), 'Category'] = 'GAMES'

df['Category'].value_counts()

GAMES                  6384
EDUCATION              3231
TOOLS                  2173
ENTERTAINMENT          1929
BOOKS_AND_REFERENCE    1518
                       ... 
DATING                   98
EVENTS                   88
BEAUTY                   85
COMICS                   74
LIBRARIES_AND_DEMO       59
Name: Category, Length: 33, dtype: int64

note: will combine more categories if necessary

## Convert 'LastUpdated' to datetime

In [19]:
from datetime import datetime

df['LastUpdated'] = df.apply(lambda row : datetime.strptime(row['LastUpdated'], '%B %d, %Y'), axis = 1)

## Clean 'Price' column

In [20]:
def convert_price(x):
    if x == '0':
        return 0
    else:
        x = float(x[1:])
    return x

df['Price'] = df.apply(lambda row : convert_price(row['Price']), axis = 1)

Check that the prices are reasonable, and remove any junk apps where necessary

In [21]:
rows = df['Price'].nlargest(n=10)
rows

1154     399.99
13934    299.99
1794      99.99
31340     84.99
4718      79.99
1785      74.99
13550     74.99
10099     54.99
27838     54.99
26564     45.99
Name: Price, dtype: float64

In [22]:
df.loc[df['Price'].isin(rows)]

,AppName,Category,Rating,Reviews,Installs,Size,Price,ContentRating,LastUpdated
1154,Plasma duct - Premium Game,GAMES,3.378378391,74.0,"1,000+",18432.0,399.99,Everyone,2019-03-04
1785,Advanced Language Therapy: Aphasia,MEDICAL,3,2.0,10+,80896.0,74.99,Everyone,2019-02-13
1794,Vargo Anesthesia Mega App,MEDICAL,4.599999905,100.0,"1,000+",35840.0,99.99,Everyone,2019-02-21
4718,Stock Screener Pro,FINANCE,3.661538363,65.0,"1,000+",1433.6,79.99,Everyone,2018-10-04
10099,Wonster Words (Pro),EDUCATION,4.416666508,12.0,100+,84992.0,54.99,Everyone,2019-03-18
13550,Language Therapy: Aphasia,MEDICAL,5,13.0,"1,000+",28672.0,74.99,Everyone,2018-11-05
13934,I AM RICH,FINANCE,5,1.0,1+,1638.4,299.99,Everyone,2019-02-09
26564,Extreme Landings Pro,GAMES,4.570792675,7190.0,"10,000+",32768.0,45.99,Everyone,2019-03-11
27838,Clinical Anesthesia Procedures,MEDICAL,4.166666508,6.0,50+,6246.4,54.99,Everyone,2019-01-03
31340,?????????????10????????????????????????,BOOKS_AND_REFERENCE,4.25,4.0,500+,2048.0,84.99,Everyone,2018-01-26


In [23]:
df.drop(df[(df['Price'] == 399.99) | (df['Price'] == 299.99) | (df['Price'] == 84.99)].index, inplace = True)

## Clean the 'Installs' column

In [24]:
df['Installs'].value_counts()

100,000+          6190
1,000,000+        4434
10,000+           4389
500,000+          2562
50,000+           2481
                  ... 
5+                  27
1+                  20
500,000,000+        13
1,000,000,000+       5
0+                   2
Name: Installs, Length: 20, dtype: int64

In [25]:
installs_0 = ['0+', '1+', '5+', '10+', '50+', '100+', '500+', '1,000+', '5,000+', '10,000+',d
df.loc[~df['Installs'].isin(installs_0), 'Installs'].value_counts()

1,000,000+        4434
500,000+          2562
10,000,000+       1490
5,000,000+        1217
50,000,000+        211
100,000,000+       140
500,000,000+        13
1,000,000,000+       5
Name: Installs, dtype: int64

In [26]:
df.loc[~df['Installs'].isin(installs_0), 'Installs'] = '>=500000'
df.loc[df['Installs'].isin(installs_0), 'Installs'] = '<500000'

In [35]:
df['Installs'].value_counts()

<500000     18726
>=500000    10072
Name: Installs, dtype: int64

## 'Ratings' and 'Review'

In [27]:
df['Rating'] = df['Rating'].astype('float')
df.describe()[['Rating']]

,Rating
count,28798.000000
mean,4.270109
std,0.509745
min,1.000000
25%,4.086033
50%,4.377251
75%,4.588889
max,5.000000


In [28]:
df.describe()[['Reviews']]

,Reviews
count,2.879800e+04
mean,6.502429e+04
std,6.520699e+05
min,1.000000e+00
25%,1.120000e+02
50%,1.127000e+03
75%,1.055150e+04
max,4.840147e+07


In [29]:
df['Category'] = df['Category'].astype('category')
df['Installs'] = df['Installs'].astype('category')
df['ContentRating'] = df['ContentRating'].astype('category')

In [30]:
df.dtypes

AppName                  object
Category               category
Rating                  float64
Reviews                 float64
Installs               category
Size                    float64
Price                   float64
ContentRating          category
LastUpdated      datetime64[ns]
dtype: object